# Battle of Neighborhoods.
Let's assume we are about to move to New York City. We've found 3 apartments at approximately same price at the following addresses:
- 49 First Ave, New York, NY 10003 
- 22 Cornelia St, New York, NY 10014
- 167 Hester St, New York, NY 10013
<br><br>
**The task is to decide which address is the most favourable to live in according to this criteria:**
- proximity to subway stations
- proximity to coffe shops 
- proximity to sushi restaurants (can't live a week without a Californa roll!)
The proximity factor is also defined as 500 meters (~1/3 mile).
<br><br>Using FourSquare data, we will identify the points of interest in the vincinity of each address, plot them on a map Folio and draw the conclusion
#### Scoring method:

|Facility   |Points|
|-----------|------|
|Subway stop|5     | 
|Coffee shop|3     | 
|Sushi Rest |2     | 


In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
# define the dataframe columns
column_names = ['Name', 'Latitude', 'Longitude'] 
addresses = ['49 1st Ave, New York, NY 10003',
             '22 Cornelia St, New York, NY 10014',
             '167 Hester St, New York, NY 10013']
# instantiate the dataframe
df_addr = pd.DataFrame(columns=column_names)
df_addr["Name"]=addresses

In [3]:
# get coordinates for each address using Nominatum
geolocator = Nominatim(user_agent="ny_explorer")
for i in range(df_addr.shape[0]):
  location = geolocator.geocode(df_addr.iloc[i]["Name"])
  df_addr.iloc[i].Latitude = location.latitude
  df_addr.iloc[i].Longitude = location.longitude
df_addr

,Name,Latitude,Longitude
0,"49 1st Ave, New York, NY 10003",40.7248,-73.9876
1,"22 Cornelia St, New York, NY 10014",40.7313,-74.0021
2,"167 Hester St, New York, NY 10013",40.7178,-73.9966


In [4]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=list(df_addr[['Latitude', 'Longitude']].iloc[0]), zoom_start=14)

# add markers to map
for addr, lat, lng in zip(df_addr['Name'], df_addr['Latitude'], df_addr['Longitude']):
    label = addr
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [ ]:
#Load FourSquare credentials
import yaml
FOURSQ_CONFIG_FILE = '4SQ.env'
with open(FOURSQ_CONFIG_FILE, 'r') as config_file:
    config = yaml.load(config_file, Loader=yaml.BaseLoader)

# set parameters for FourSquare API calls
CLIENT_ID = config['FOURSQUARE']['CLIENT_ID']
CLIENT_SECRET = config['FOURSQUARE']['CLIENT_SECRET']
VERSION = '20190708' # Foursquare API version
radius = 500 # 1KM radius
limit = 500 # 500 result limit 

In [ ]:
def getNearbyVenues(coords, categoryID='', q_name='',radius=500, intent="browse"):
    venues_list=[]
    for address, lat, lng in coords:
        #print(q_name)
            
        # create the API request URL
        url = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}'
        '&intent={}&categoryId={}'
        '&limit={}'
        '&query={}'
        ).format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            intent,
            categoryID,
            limit,
            q_name
        )
            
        # make the GET request
#        print("URL:"+url)
        url_results = requests.get(url).json()["response"]
#        print(url_results)
        #results=url_results['groups'][0]['items']
        results=url_results['venues']
        # return only relevant information for each nearby venue
        venues_list.append([(
 #           v['id'],
            address,
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']  
#            v['categories'][0]['name']
        )
            for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [ 
#                  'Venue ID',
                  'Address',
                  'Name', 
                  'Latitude', 
                  'Longitude' 
#                  'Venue Category'
    ]
    
    return(nearby_venues)

In [ ]:
# declare foursquare categories 
fsq_cat={'Coffee Shop':'4bf58dd8d48988d1e0931735',
         'Subway':'4bf58dd8d48988d1fd931735',
         'Sushi Restaurant':'4bf58dd8d48988d1d2941735'}

In [ ]:
#get coffee shops
df_coffee = getNearbyVenues(df_addr.values, categoryID=fsq_cat["Coffee Shop"],radius=500)
df_coffee.head(5)

,Address,Name,Latitude,Longitude
0,"49 1st Ave, New York, NY 10003",Mudpark,40.723127,-73.988794
1,"49 1st Ave, New York, NY 10003",Allegro Coffee Roasters,40.723627,-73.991291
2,"49 1st Ave, New York, NY 10003",Ludlow Coffee Supply,40.721711,-73.987521
3,"49 1st Ave, New York, NY 10003",Mudspot,40.729082,-73.986750
4,"49 1st Ave, New York, NY 10003",The Bean,40.725762,-73.989652


In [ ]:
#get subway stations
df_subway = getNearbyVenues(df_addr.values, categoryID=fsq_cat["Subway"],radius=500)
df_subway.head(5)

,Address,Name,Latitude,Longitude
0,"49 1st Ave, New York, NY 10003",MTA Subway - 2nd Ave (F),40.723101,-73.988866
1,"22 Cornelia St, New York, NY 10014",MTA Subway - W 4th Street/Washington Square (A...,40.731113,-74.001307
2,"22 Cornelia St, New York, NY 10014",MTA Subway - Houston St (1),40.728178,-74.005370
3,"22 Cornelia St, New York, NY 10014",Christopher Street PATH Station,40.732937,-74.006905
4,"22 Cornelia St, New York, NY 10014",MTA Subway - Christopher St/Sheridan Square (1),40.733619,-74.002823


In [ ]:
#get Sushi restaurants (yum!)
df_sushi = getNearbyVenues(df_addr.values, categoryID=fsq_cat["Sushi Restaurant"],radius=500)
df_sushi.head(5)

In [ ]:
# Assign location type and merge the data sets 
df_coffee["Type"]="Coffee"
df_subway["Type"]="Subway"
df_sushi["Type"]="Sushi"
df_all = pd.concat([df_coffee, df_subway, df_sushi])
df_all.head(5)

In [ ]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=list(df_addr[['Latitude', 'Longitude']].iloc[0]), zoom_start=14)
chain_color = { "Address":"red","Coffee":"yellow","Subway":"green","Sushi":"blue"}

#add addresses to the map
for lat, lng, name in zip(df_addr['Latitude'], df_addr['Longitude'], df_addr['Name']):
    label = 'Address:' + name 
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  

# add markers to map
for lat, lng, name, ltype in zip(df_all['Latitude'], df_all['Longitude'], df_all['Name'],df_all['Type']):
    label = ltype +':' + name 
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=chain_color.get(ltype),
        fill=True,
        fill_color=chain_color.get(ltype),
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [ ]:
#get the total number of venues by address
df_score = pd.concat([df_all["Address"],pd.get_dummies(df_all.Type)],1).groupby(['Address']).sum().reset_index()
df_score

Finally, we are ready to apply the scoring method and anounce the winner.

In [ ]:
df_score["Points"]=df_score["Coffee"]*3+df_score['Subway']*5+df_score['Sushi']*2
df_score.sort_values(by='Points', ascending=False, inplace = True)
df_score

In [ ]:
#Announce the winner
print("And the winner is..."+df_score.iloc[0]["Address"]+'!')